## Homework

In [ ]:
import math
from ROOT import *
import numpy as np

# For inline image display
from IPython.display import Image

In [ ]:
# Set ROOT graphics options

gROOT.SetStyle("Plain")

# Default canvas  size

gStyle.SetCanvasDefW(600)
gStyle.SetCanvasDefH(400)
 
# set the margins
  
gStyle.SetPadBottomMargin(0.18)
gStyle.SetPadTopMargin(0.08)
gStyle.SetPadRightMargin(0.08)
gStyle.SetPadLeftMargin(0.17)

# set the number of divisions to show

gStyle.SetNdivisions(510, "xy")

# show grid

gStyle.SetPadGridX(1)
gStyle.SetPadGridY(1)

# border ploted only around statistics

gStyle.SetPadBorderMode(0)
gStyle.SetFrameBorderMode(0)
gStyle.SetLegendBorderSize(0)
gStyle.SetTitleBorderSize(0)
gStyle.SetStatBorderSize(1)

# Text sizes

gStyle.SetTextSize(0.05)
gStyle.SetTitleSize(0.06,"xyz")
gStyle.SetLabelSize(0.05,"xyz")

gStyle.SetLabelOffset(0.015,"xyz")
gStyle.SetTitleOffset(1.2,"yz")
gStyle.SetTitleOffset(1.0,"x")

# Font settings

font = 42
gStyle.SetTitleFont(font)
gStyle.SetTitleFontSize(0.05)
gStyle.SetStatFont(font)
gStyle.SetStatFontSize(0.06)
gStyle.SetTextFont(font)
gStyle.SetLabelFont(font,"xyz")
gStyle.SetTitleFont(font,"xyz")
gStyle.SetLegendFont(font)

# Line settings

gStyle.SetHistLineWidth(2)
gStyle.SetFrameLineWidth(2)
gStyle.SetFuncWidth(2)
gStyle.SetHistLineColor(kBlue)
gStyle.SetFuncColor(kRed)

# Log scale
gStyle.SetOptLogy(0)    # 0 no log y scale, 1 log y scale

# Histogram title
gStyle.SetOptTitle(1)  

# Histogram statistics
gStyle.SetOptStat(1100)

# Fit results
gStyle.SetOptFit(0) 


Assuming that the number of counted particles is proportional to the energy, the desired calorimeter depth can be found by integrating a function that fits a given cascade profile. If 99 percent of the particles are found in a given range, then that range can be considered to contain 99 percent of the energy approximately. The upper limit of the integration range is found by comparing a given integral value to the integral for the full range. 

The program will generate plots and .txt files with information about calorimeter depth for 5 different energies.

In [ ]:
# Class for single shower particle

class shParticle:
    'Particle of the EM shower (in 1D)'
    E = 0.0     # initial energy
    q = 0       # charge
    xmin = 0.0
    xmax = 0.0
    
    def __init__(self,E,q,xmin):
        self.E = E
        self.q = q
        self.xmin = xmin
        self.xmax = xmin
        
    def __str__(self):
        if self.xmin == self.xmax :
            return 'E = %g   q = % g    created at X = %g [X0]' % (self.E, self.q, self.xmin)
        else :
            return 'E = %g   q = % g    propagated from X = %g to %g  [X0]' % (self.E, self.q, self.xmin, self.xmax)
        
    def setlen(self,shlen):     # define endpoint of the particle; length after it decays
        self.xmax = self.xmin + shlen
 
    def getlen(self):       # define start of the particle
        return self.xmax - self.xmin
    
    def getsamp(self,dX):   # tells how long the track of the particle is in quantized manner
        i1 = int(self.xmin/dX)
        i2 = int(self.xmax/dX)
        return i2-i1


In [ ]:
# EM shower class

class emShower:
    'Model of EM shower development'

    def __init__(self,E0,q0,x0):
        # initialize with input particle
        self.shower = []    # collection of particles
        self.shower.append(shParticle(E0,q0,x0))
        # only propagated particles are counted
        # so set counters to zero at that point
        self.npar = 0  
        self.nch = 0    # number of charged particles
        self.chlen = 0.0
        self.eloss = 0.0

    def __str__(self):
        return 'Shower with %d particles, %d charged, total track lenght %g X0, energy loss %g GeV' % (self.npar, self.nch, self.chlen, self.eloss)
    
    def develope(self,Ec,dEdx=0.0):
        # consider increasing number of particles in the shower
        for par in self.shower :
            # conversion/radiation length [X0]
            if par.q == 0 :     # check if it is charged
                intlen = 9.0/7.0
            else :  
                intlen = 1.0    # electron positron
               
            # generate conversion/radiation point
            
            shlen = np.random.exponential(intlen)
            
            # check energy loss of particle
            
            elen = 0
            
            if par.q != 0 and dEdx > 0 :
                elen = shlen * dEdx
                if elen > par.E :
                    shlen = par.E/dEdx
                    elen = par.E
            
            # propagate particle to this point
            
            par.setlen(shlen)
            self.npar+=1
            if par.q != 0 :
                self.nch+=1
                self.chlen+=shlen
                self.eloss+=elen

            # Energy at the final point
            
            Eleft = par.E - elen
            
            # Final point - origin for new particles
            
            xnew = par.xmax 
           
            # If above critical energy:
            #   convert gamma to two photons or radiate photon
            if Eleft > Ec :
                E1 = Eleft * np.random.random(1)    #  Very siplified energy splitting
                E2 = Eleft - E1
                if par.q == 0 :     # if it is photons
                    self.shower.append(shParticle(E1,+1,xnew))
                    self.shower.append(shParticle(E2,-1,xnew))
                else:       # electron positron
                    self.shower.append(shParticle(E1,par.q,xnew))
                    self.shower.append(shParticle(E2,0,xnew))
                    
    def sample(self,Xsamp):     # calculate how many samples are from the shower
        nsamp = 0
        for par in self.shower:
            if par.q != 0 :
                nsamp += par.getsamp(Xsamp)
        return nsamp

    def profile(self,Nbin,Xmax):        # shape of the shower as the histogram
        hprof = TH1D("hprof","Shower profile",Nbin, 0, Xmax)
        hprof.GetXaxis().SetTitle("x [X0]")
        hprof.GetYaxis().SetTitle("Particles")
        dX=Xmax/Nbin
        for par in self.shower:
            if par.q != 0 :
                nx = par.getsamp(dX)
                for ix in range(nx):
                    hprof.Fill(par.xmin+ix*dX)
        return hprof
    
    def dump(self,Nprt):
        for iprt in range(Nprt):
            print(self.shower[iprt])
            

In [ ]:
#  Incident particle

energies = np.array([1, 10, 50 ,75, 100])

for e in energies:

    E0 = e
    q0 = -1

    # Initialise shower with incident particle

    myshower = emShower(E0,q0,0.)

    print(myshower.shower[0])

    # Critical energy [GeV] for Copper

    Ec = 0.020

    # Ionization losses [GeV/X0] for Copper (MIP)

    dEdx = 0.018

    # Execute shower development procedure

    myshower.develope(Ec,dEdx)

    # Test shower dump (first 10 rows)

    myshower.dump(20)      # create particles in shower
    print(myshower)         # summary

    # Other useful  information about our shower
    # Direct access to shower information
    # Can be used to model response of the uniform calorimeter

    print('Shower with ',myshower.npar,' particles, ',myshower.nch,' charged particles')
    print('Total length of charged particle tracks: ',myshower.chlen)
    # Repeat shower simulation multiple times
    # work

    avprofile=shprofile
    avprofile.SetNameTitle("havprof","Average shower profile " + str(E0) + ' GeV')
    avprofile.Reset()

    Nmc = 1000
    myshowers = []

    for imc in range(Nmc):
        newshower = emShower(E0,q0,0.1)
        newshower.develope(Ec,dEdx)
        myshowers.append(newshower)
        if (imc+1)%100 == 0:
            print(imc+1,' showers generated ',myshowers[imc].npar,' particles')
    # Average generated showers

    wmc = 1/Nmc

    for newshower in myshowers:
        newprofile=newshower.profile(Nprof,Xmax)
    #    newprofile=myshowers[imc].profile(Nprof,Xmax)
        avprofile.Add(newprofile,wmc)
        newprofile.Delete()

    plotname = 'avprofile' + str(E0) + '.png'
    csh2 = TCanvas('csh2')

    avprofile.DrawCopy()
    # avprofile.Fit(fitfunc)

    csh2.Update()
    csh2.Print(plotname)

    Image(filename=plotname)

    class myfunc:   # fit function as a class
        def __call__( self, t, par ):
            x0 = par[0] # x center
            y0 = par[1] #y center
            sigma = par[2] #sigma
            x = t[0]   
            tmp = y0 * np.exp(- (x0 / sigma) ** 2 * (((x - x0) / x0) - np.log(x / x0)))
            return tmp

    fitrangestart = 0.1   
    fitrangeend = 50

    parN = 3    # number of fit parameters 

    pycallable = myfunc()

    func = TF1("func", pycallable, fitrangestart, fitrangeend, parN)    # initialization of fitting function
    func.SetParameters(11, 60, 6)     # setting parameters for our function

    avprofile.Fit(func, 'Q')
    # 

    fitresult = func.GetParameters()

    leg=TLegend(0.7,0.8,0.9,0.9)
    leg.AddEntry(avprofile,"Shower data")
    leg.AddEntry(func,"Fitting function")
    leg.Draw()


    csh2.Update()
    csh2.Print(plotname)
    # 
    Image(filename=plotname)

    print("Fit results: \n X maximum position: " + str(fitresult[0]) + "\n Y maximum position: " + str(fitresult[1]) + "\n Sigma: " + str(fitresult[2]))


    import matplotlib.pyplot as plt
    from scipy import integrate

    def intfunc(x, x0, y0, sigma):
        return y0 * np.exp(- (x0 / sigma) ** 2 * (((x - x0) / x0) - np.log(x / x0)))

    start = 1
    end = 50

    x = np.linspace(start, end, 100)
    x0 = fitresult[0]
    y0 = fitresult[1]
    sigma = fitresult[2]

    y = intfunc(x, x0, y0, sigma)
    xreversed = np.flip(x)

    whole_int = integrate.quad(intfunc, start, end,  args=(x0, y0, sigma))   # whole integration of intfunc

    # here I check in which range 99 percent of the energy is contained 
    for i in xreversed:
        integral = integrate.quad(intfunc, start, i,  args=(x0, y0, sigma))
        ratio = integral[0] / whole_int[0]
        if ratio < 0.99:
            print(i, ratio)

            infoname = 'calor-depth-info' + str(E0) + '.txt'
            # generate .txt file with information about contained energy and calorimeter depth
            with open(infoname, 'w') as file:
                file.write(str(i) + ' depth, energy containted in percent: ' + str(ratio))
            break